In [1]:
import pandas as pd
import pickle

df_files = pd.read_excel('../data/patentData_80000_20180622.xlsx', encoding='utf-8')


In [2]:
import jieba
stopwordfile = open('../dict/StopWords_CON.txt', 'r', encoding='utf-8')
def getstopword():
    w = set()
    for line in stopwordfile:
        line.strip().split('\n')
        w.add(line[:len(line)-1].strip())
    return w

stopwordset = getstopword()

def cutWords(sentence):
    word_list = jieba.cut(sentence)
    res = ' '.join(word_list)
    res = res.split(' ')
    tempX = ''
    for i in res:
        if i not in stopwordset:
            tempX+=i
            tempX+=' '
    return tempX.strip()

In [3]:
df_files['all_sentence'] = df_files['标题'] + df_files['摘要'] + df_files['首项权利要求']


In [5]:
import numpy as np
def DropNan(sentence):
    if sentence is np.nan:
        return ''
    return sentence

df_files['all_sentence'] = df_files['all_sentence'].apply(DropNan)
df_files['all_sentence'] = df_files['all_sentence'].apply(lambda x: x.replace('\n', ' '))
df_files['all_sentence'] = df_files['all_sentence'].apply(cutWords)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built succesfully.


In [6]:
word2id, id2word = pickle.load(open("../data/word2idandid2word.pkl","rb"))
def WordToId(sentence):
    res = []
    word_list = sentence.split(' ')
    for word in word_list:
        if word not in word2id.keys():
            res.append(1)
        else:
            res.append(word2id[word])
    return res

In [7]:
df_files['all_sentence'] = df_files['all_sentence'].apply(WordToId)


In [10]:
X_all = df_files['all_sentence'].values

In [11]:
X_all_padding_len = 200

from keras.preprocessing.sequence import pad_sequences
X_all = pad_sequences(X_all, maxlen=X_all_padding_len, padding='pre', truncating='post')

print (X_all.shape)

(80000, 200)


In [14]:
def LabelDropNan(label):
    if label is np.nan:
        print (label)
        return str(-1)
    return str(label)
y_str = df_files['主IPC分类号-小类'].apply(LabelDropNan).values


In [19]:
label2id = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6,
    'H': 7
}

id2label = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H'
}

In [20]:
pickle.dump([label2id, id2label], open("../data/label2idandid2label.pkl","wb"))

In [21]:
y_onehot = np.eye(8, dtype=int)


In [22]:
y_onehot

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [24]:
y = []
for i in y_str:
    y.append(y_onehot[label2id[i[0]]])
y = np.array(y)

In [25]:
print (y.shape)

(80000, 8)


In [26]:
print (X_all.shape)

(80000, 200)


In [27]:
pickle.dump(X_all, open("../data/X_data.pkl","wb"))
pickle.dump(y, open("../data/y_data.pkl","wb"))
